In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [3]:
eligibility = client.to_pandas("""
    select
        p.project_id,
        p.project_name,
        p.display_name,
        e.transaction_count,
        e.active_days,
        e.active_addresses_count,
        e.gas_fees,
        e.is_eligible,
        e.has_defillama_adapter,
        e.has_bundle_bear
    from oso.int_superchain_s7_onchain_builder_eligibility e
    join oso.projects_v1 p on e.project_id = p.project_id
    join oso.projects_by_collection_v1 pbc on p.project_id = pbc.project_id
    where pbc.collection_name = '8-1'
    order by e.transaction_count desc
""")
eligibility.tail(1)

,project_id,project_name,display_name,transaction_count,active_days,active_addresses_count,gas_fees,is_eligible,has_defillama_adapter,has_bundle_bear
186,ZcxGuOFKrVdSeP7Ej7qb9S1bs4waWJMHTjpNEOP+tq0=,0xbbb2ce15e5047267dcd0f25775dde250f27f9828a305...,Babylonians,0,0,0,0.0,<NA>,False,False


In [4]:
df_metrics = client.to_pandas("""
    select
        m.project_id,
        p.display_name,
        pbc.project_name,
        m.chain,
        m.metric_name,
        DATE_FORMAT(m.sample_date, '%Y-%m-%d') AS sample_date,
        DATE_FORMAT(m.sample_date, '%b %Y') AS measurement_period,
        m.amount
    from oso.int_superchain_s7_onchain_metrics_by_project m
    join oso.projects_by_collection_v1 pbc on m.project_id = pbc.project_id
    join oso.projects_v1 p on pbc.project_id = p.project_id
        where pbc.collection_name = '8-1'
        and m.sample_date between date('2025-01-01') and date('2025-02-28')
""")
df_metrics.tail(1)

,project_id,display_name,project_name,chain,metric_name,sample_date,measurement_period,amount
3450,dk1gwy0G/dP9SOcyolL+bfgPQJ0ngqXF6ro9qyQdGwo=,Air3,0xf9ebb0464e7162627ab3de5fe03a21c879e932233e14...,ZORA,amortized_contract_invocations_monthly,2025-01-01,Jan 2025,313.2


In [5]:
(df_metrics.query("metric_name == 'average_tvl_monthly'").pivot_table(
    index='display_name',
    columns='sample_date',
    values='amount',
    aggfunc='sum'
) / 1_000_000).sort_values(by='2025-02-01', ascending=False)

sample_date,2025-01-01,2025-02-01
display_name,,
Aerodrome Finance,1245.533253,943.595737
Aave,571.364921,600.653132
Uniswap on Superchain (Oku),698.287355,480.403061
Morpho,459.004645,479.616796
Moonwell,178.625471,144.383072
Stargate Finance,76.937209,143.410985
Bedrock on Optimism Superchain,142.024507,135.238943
Compound Finance,118.35966,110.635796
Extra Finance,140.948681,93.446806


In [6]:
DIR = "../../../../../../../../GitHub/Retro-Funding/"
#eligibility.to_csv(DIR + "eval-algos/S7/data/M1/onchain/onchain_builders.csv", index=False)
#df_metrics.to_csv(DIR + "eval-algos/S7/data/M1/onchain/onchain_metrics_by_project.csv", index=False)

In [7]:
df_rewards = pd.read_csv(
    DIR + "eval-algos/S7/data/M1/onchain/onchain_builders_m1_goldilocks_rewards.csv"
)
df_eligibility_dump = eligibility.merge(
    df_rewards[['project_name', 'op_reward']],
    left_on='project_name',
    right_on='project_name',
    how='outer'
)
df_eligibility_dump['is_eligible'] = df_eligibility_dump['is_eligible'].fillna(False)
df_eligibility_dump['round_id'] = '8'
cols = [
    'project_id', 'project_name', 'display_name', 
    'round_id',  'is_eligible', 'op_reward',
    'transaction_count', 'active_days', 'active_addresses_count',
    'gas_fees', 'has_defillama_adapter', 'has_bundle_bear'
]
df_eligibility_dump = df_eligibility_dump[cols]

json_str = df_eligibility_dump.to_json(orient='records', indent=2)
clean_json_str = json_str.replace('\\/', '/')
with open('m1_onchain_builders_eligibility.json', 'w') as file:
    pass
    #file.write(clean_json_str)